In [ ]:
import networkx as nx
import numpy as np
# Load the graph
from xlron.environments.rsa import make_graph

graph = make_graph("nsfnet", topology_directory="/Users/michaeldoherty/git/XLRON/topologies")

# Make N x N numpy matrix where N is the number of nodes, with values 1/N
traffic_matrix = np.full((len(graph.nodes), len(graph.nodes)), 1 / len(graph.nodes))


In [10]:
node_cuts = nx.all_node_cuts(graph)

{4, 6, 8}

In [20]:
len(list(node_cuts))

9

In [ ]:
import networkx as nx
import itertools
import random

def find_all_cut_sets(G):
    """
    Find all cut-sets of a graph G.
    Returns a list of cut-sets, where each cut-set is represented as a set of edges.
    """
    cut_sets = []
    for i in range(1, len(G.nodes) // 2 + 1):
        for subset in itertools.combinations(G.nodes, i):
            complement = set(G.nodes) - set(subset)
            if nx.is_connected(G.subgraph(subset)) and nx.is_connected(G.subgraph(complement)):
                cut_set = set(nx.edge_boundary(G, subset))
                cut_sets.append(cut_set)
    return cut_sets

def estimate_congestion_level(G, cut_set, traffic_matrix):
    """
    Estimate the congestion level of a cut-set based on the traffic matrix.
    Returns the ratio of the number of links in the cut-set to the expected number of lightpaths traversing the cut-set.
    """
    subset = set(nx.node_connected_component(G.edge_subgraph(cut_set), next(iter(cut_set))[0]))
    complement = set(G.nodes) - subset
    expected_lightpaths = sum(traffic_matrix[i][j] for i in subset for j in complement)
    return len(cut_set) / expected_lightpaths

def estimate_capacity_upper_bound(G, traffic_matrix, num_wavelengths, num_cut_sets):
    """
    Estimate the capacity upper bound using the cut-set analysis method.
    Returns the estimated upper bound on the number of lightpaths that can be accommodated.
    """
    cut_sets = find_all_cut_sets(G)
    congestion_levels = [(cut_set, estimate_congestion_level(G, cut_set, traffic_matrix)) for cut_set in cut_sets]
    congestion_levels.sort(key=lambda x: x[1])
    
    lightpaths = 0
    wavelength_availability = {link: set(range(num_wavelengths)) for link in G.edges}
    
    while True:
        src, dst = random.choices(list(G.nodes), k=2)
        cut_sets_to_traverse = [cut_set for cut_set, _ in congestion_levels[:num_cut_sets] if 
                                any(src in set(nx.node_connected_component(G.edge_subgraph(cut_set), node)) and 
                                    dst in set(G.nodes) - set(nx.node_connected_component(G.edge_subgraph(cut_set), node))
                                    for node in cut_set)]
        
        if not cut_sets_to_traverse:
            lightpaths += 1
            continue
        
        available_wavelengths = set.intersection(*(wavelength_availability[link] for cut_set in cut_sets_to_traverse for link in cut_set))
        
        if not available_wavelengths:
            break
        
        wavelength = available_wavelengths.pop()
        for cut_set in cut_sets_to_traverse:
            for link in cut_set:
                wavelength_availability[link].remove(wavelength)
        
        lightpaths += 1
    
    return lightpaths

# Example usage
G = nx.Graph()
G.add_edges_from([('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('E', 'F'), ('A', 'F')])

traffic_matrix = {node: {other: random.randint(1, 10) for other in G.nodes if other != node} for node in G.nodes}

num_wavelengths = 10
num_cut_sets = 3

upper_bound = estimate_capacity_upper_bound(G, traffic_matrix, num_wavelengths, num_cut_sets)
print(f"Estimated capacity upper bound: {upper_bound} lightpaths")